# Load Testing Data

In [1]:
import pandas as pd
import numpy as np
import os
import pysam

In [2]:
input_sam_file = 'data/small-cell-sorted.bam'

In [3]:
def parse_record(record):
    """line parser to build dataframe, supports missing tags in test data"""
    data = {
        'qname': record.query_name,
        'flag': record.flag,
        'reference': record.reference_id,
        'position': record.pos,
        'mapq': record.query_qualities,
        'cigar': record.cigarstring,
        'rnext': record.rnext, 
        'pnext': record.pnext,
        'tlen': record.tlen, 
        'sequence': record.seq,
        'quality': record.qual,
    }
    for name, tag in record.get_tags():
        data[name] = tag
    return pd.Series(data)

with pysam.AlignmentFile(input_sam_file, 'rb') as f:
    records = []
    for record in f:
        records.append(parse_record(record))

data = pd.concat(records, axis=1).T

results_scalar = {}

/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:23: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


# Build Expectations for Testing Data

## Number of Reads

In [4]:
results_scalar['n_reads'] = len(data)
print(results_scalar['n_reads'])

656


## Number of Genes

In [5]:
results_scalar['genes_detected'] = len(data.groupby(['GE']))
print(results_scalar['genes_detected'])

11


Gene table should have 8 entries plus a header for a total of 9 lines

## Number of Molecules

Molecules are defined as a unique triplet of CB, UB, and GE

In [6]:
results_scalar['n_molecules'] = len(data.groupby(['CB', 'UB', 'GE']))
print(results_scalar['n_molecules'])

249


## Number of Fragments

Fragments are defined as molecules are (CB, UB, GE) but must additionally have a unique position

In [7]:
results_scalar['n_fragments'] = len(data.groupby(['CB', 'UB', 'GE', 'position']))
print(results_scalar['n_fragments'])

499


## Most Abundant Gene

Based on the above, at least one of the genes has to be observed more than once. Which is it? 

In [8]:
results_scalar['most_abundant'] = data.groupby(['GE']).size().idxmax().split(':')[-1]
results_scalar['most_abundant_gene_n_observations'] = data.groupby(['GE']).size().max()
print(results_scalar['most_abundant'], results_scalar['most_abundant_gene_n_observations'])

MTATP6P1 300


In [9]:
results_scalar['perfect_molecule_barcodes'] = 0
for c, r in zip(data['UB'], data['UR']):
    if c.split(':')[-1] == r.split(':')[-1]:
        results_scalar['perfect_molecule_barcodes'] += 1

Calculate the alignment metrics

In [10]:
results_scalar['reads_mapped_exonic'] = sum(data['XF'] == 'CODING')

In [11]:
results_scalar['reads_mapped_intronic'] = sum(data['XF'] == 'INTRONIC')

In [12]:
results_scalar['reads_mapped_utr'] = sum(data['XF'] == 'UTR')

In [13]:
results_scalar['reads_mapped_uniquely'] = sum(data['NH'] == 1)

In [14]:
results_scalar['duplicate_reads'] = sum((data['flag'] & 1024).astype(bool))

In [15]:
results_scalar['spliced_reads'] = sum(1 for v in data['cigar'] if 'N' in v)

Calculate the higher-order metrics

In [16]:
calc_func_fraction = lambda x: sum(1 for c in x.split(':')[-1] if ord(c) > 63) / len(x.split(':')[-1])
calc_func_mean = lambda x: np.mean([ord(c) - 33 for c in x.split(':')[-1]])

data['num_UY_qual_fraction'] = data['UY'].apply(calc_func_fraction)

data['num_base_qual_fraction'] = data['quality'].apply(calc_func_fraction)
data['num_base_qual_mean'] = data['quality'].apply(calc_func_mean)

grouped_by_gene = data.groupby(['GE'])

In [17]:
results_series = {}

In [18]:
# vector values
# I changed these to retain the index to make merging into a dataframe easier, and guarantee same order. 
results_series['molecule_barcode_fraction_bases_above_30_mean'] = grouped_by_gene.mean()['num_UY_qual_fraction']
results_series['molecule_barcode_fraction_bases_above_30_variance'] = grouped_by_gene.var()['num_UY_qual_fraction']

results_series['genomic_reads_fraction_bases_quality_above_30_mean'] = grouped_by_gene.mean()['num_base_qual_fraction']
results_series['genomic_reads_fraction_bases_quality_above_30_variance'] = grouped_by_gene.var()['num_base_qual_fraction']
results_series['genomic_read_quality_mean'] = grouped_by_gene.mean()['num_base_qual_mean']
results_series['genomic_read_quality_variance'] = grouped_by_gene.var()['num_base_qual_mean']

reads_per_gene = data.groupby(['GE']).size()

In [19]:
molecules_per_gene = grouped_by_gene.apply(lambda x: len(x.groupby(['UB', 'CB']).size()))
fragments_per_gene = grouped_by_gene.apply(lambda x: len(x.groupby(['UB', 'CB', 'position']).size()))
reads_per_molecule = reads_per_gene / molecules_per_gene
reads_per_fragment = reads_per_gene / fragments_per_gene
fragments_per_molecule = fragments_per_gene / molecules_per_gene
results_series['reads_per_molecule'] = reads_per_molecule
results_series['reads_per_fragment'] = reads_per_fragment
results_series['fragments_per_molecule'] = fragments_per_molecule

# scalar values
results_scalar['fragments_with_single_read_evidence'] = np.sum(data.groupby(['CB', 'UB', 'GE', 'position']).size() == 1)
results_scalar['molecules_with_single_read_evidence'] = np.sum(data.groupby(['CB', 'UB', 'GE']).size() == 1)

In [20]:
pd.DataFrame(results_series)

,fragments_per_molecule,genomic_read_quality_mean,genomic_read_quality_variance,genomic_reads_fraction_bases_quality_above_30_mean,genomic_reads_fraction_bases_quality_above_30_variance,molecule_barcode_fraction_bases_above_30_mean,molecule_barcode_fraction_bases_above_30_variance,reads_per_fragment,reads_per_molecule
GE,,,,,,,,,
AL669831.4,4.000000,37.224490,6.461544,0.892857,0.008087,1.000000,0.000000,1.000000,4.000000
HES4,10.750000,35.059971,23.844938,0.802900,0.036894,0.985965,0.002299,1.325581,14.250000
ISG15,4.538462,34.151541,27.507810,0.767803,0.042226,0.984043,0.002861,1.593220,7.230769
LINC01128,1.500000,38.860544,1.547931,0.955782,0.000659,1.000000,0.000000,1.000000,1.500000
MTATP6P1,1.496894,34.308469,26.048876,0.777279,0.041522,0.984667,0.005584,1.244813,1.863354
MTCO1P12,1.035714,33.809215,27.735367,0.761286,0.046452,0.993939,0.000587,1.137931,1.178571
MTCO2P12,1.000000,21.693878,NaN,0.234694,NaN,1.000000,NaN,1.000000,1.000000
MTCO3P12,1.000000,26.224490,NaN,0.438776,NaN,1.000000,NaN,1.000000,1.000000
MTND1P23,1.200000,25.280612,12.735975,0.423469,0.023188,0.950000,0.007000,1.000000,1.200000


# Write Results to File for Automated Testing

In [21]:
pd.Series(results_scalar).to_csv('%s_testing_knowledge_scalar.csv' % input_sam_file.replace('.bam', ''))
pd.DataFrame(results_series).to_csv('%s_testing_knowledge_series.csv' % input_sam_file.replace('.bam', ''))

In [22]:
# do a comparison of the whole 2d dataframe at once
np.allclose(
    pd.DataFrame(results_series).fillna(0).values,  # fill nans with zero, call values to get the numpy array the dataframe is based on
    pd.read_csv('%s_testing_knowledge_series.csv' % input_sam_file.replace('.bam', ''), index_col=0, header=0).fillna(0).values
)

True

In [23]:
# to get most_abundant alone: 

In [24]:
test_read_scalar = pd.read_csv('%s_testing_knowledge_scalar.csv' % input_sam_file.replace('.bam', ''),
                               index_col=0, header=None, squeeze=True)

# extract this, we're going to drop it from the array to do some conversion to numeric
most_abundant = test_read_scalar['most_abundant'] 

# drop most abundant, convert to float, fill any NaN values with 0, and call .values to get the numpy array pandas objects are based on.
for_comparison = test_read_scalar.drop('most_abundant').astype(float).fillna(0).values


# note, have to drop the string value and convert to float before this works. 
np.allclose(
    pd.Series(results_scalar).drop('most_abundant').fillna(0).values,  # do the same thing as above to the one in memory
    for_comparison
)

True

In [25]:
# get a metric from a dataframe: 
df = pd.DataFrame(results_series)
df['genomic_read_quality_mean']

GE
AL669831.4    37.224490
HES4          35.059971
ISG15         34.151541
LINC01128     38.860544
MTATP6P1      34.308469
MTCO1P12      33.809215
MTCO2P12      21.693878
MTCO3P12      26.224490
MTND1P23      25.280612
MTND2P28      32.570748
NOC2L         35.627943
Name: genomic_read_quality_mean, dtype: float64

In [26]:
# get a numpy array from the dataframe
compare_me = df['genomic_read_quality_mean'].values

In [27]:
# compare two numpy arrays that are slightly different
eps = np.random.rand(11) * 1e-8
np.allclose(compare_me, compare_me + eps)


True

In [28]:
# it is actually discriminative, though
np.allclose(compare_me, np.arange(11))

False

# Look at the metrics output

In [29]:
import tempfile
from sctools.metrics.gatherer import GatherGeneMetrics, GatherCellMetrics
from itertools import product

In [30]:
_data_dir = './data'
_gene_sorted_bam = _data_dir + '/small-gene-sorted.bam'
_cell_sorted_bam = _data_dir + '/small-cell-sorted.bam'

_test_dir = tempfile.mkdtemp()
_gene_metric_output_file = _data_dir + '/gene_metrics.csv'
_cell_metric_output_file = _data_dir + '/cell_metrics.csv'
_scalar_gene_testing_knowledge = pd.read_csv(
    _data_dir + '/small-gene-sorted_testing_knowledge_scalar.csv', index_col=0, squeeze=True,
    header=None)
_scalar_cell_testing_knowledge = pd.read_csv(
    _data_dir + '/small-cell-sorted_testing_knowledge_scalar.csv', index_col=0, squeeze=True,
    header=None)


cell_gatherer = GatherCellMetrics(_cell_sorted_bam, _cell_metric_output_file)
cell_gatherer.extract_metrics()
_cell_metrics = pd.read_csv(_cell_metric_output_file, index_col=0)

gene_gatherer = GatherGeneMetrics(_gene_sorted_bam, _gene_metric_output_file)
gene_gatherer.extract_metrics()
_gene_metrics = pd.read_csv(_gene_metric_output_file, index_col=0)

parameters = [
    ['n_genes', int, np.sum],
    ['n_molecules', int, None],
    ['n_fragments', int, np.sum],
    # ('most_abundant', str, lambda x: x.index[np.argmax(x, 0]]],
    ['most_abundant_gene_n_observations', int, np.max],
    ['perfect_molecule_barcodes', int, np.sum],
    ['reads_mapped_exonic', int, np.sum],
    ['reads_mapped_intronic', int, np.sum],
    ['reads_mapped_utr', int, np.sum],
    ['reads_mapped_uniquely', int, np.sum],
    ['duplicate_reads', int, np.sum],
    ['spliced_reads', int, np.sum],
]

metrics = [
    [_cell_metrics, _scalar_cell_testing_knowledge],
    [_gene_metrics, _scalar_gene_testing_knowledge],
]

test_combinations = [m + p for m, p in product(metrics, parameters)]